In [57]:
import tensorflow as tf

In [58]:
import numpy as np
import pandas as pd

In [59]:
import trafficDataUtility as util

In [60]:
path = "C:\\Users\\KangYounKook\\Desktop\\ML_Study\\"

In [61]:
filename = "sectionInfo.txt"

In [62]:
dataset = {}
dataset = util.getDatasetFromFile(path+"sectionInfo.txt")

getDataFromFile  C:\Users\KangYounKook\Desktop\ML_Study\sectionInfo.txt


In [63]:
filename = "PeriodTotalAverage.txt"
util.updateDatasetFromFile(path+filename, dataset)

loadDataFromFile  C:\Users\KangYounKook\Desktop\ML_Study\PeriodTotalAverage.txt
update Dataset [move_count,work_count,ratio]


In [98]:
classes = 10

In [111]:
filename = "input" + str(classes) +"_all.csv"
#filename = "input10_all.csv"
data = util.loadCsvInput(path,filename)

loadCsvInput  C:\Users\KangYounKook\Desktop\ML_Study\input10_all.csv


In [112]:
# model

In [113]:
outfilename = "DNN_Result" + str(classes) + ".txt"
f = open(outfilename,'w')

In [114]:
#selectedList = dataset.keys()
selectedList = ['Section697','Section754','Section1212','Section852']

In [115]:
def checkCondition(id,data,dataset):
    if id not in data.columns:
        return False
    if dataset[id][5] < 1440:
        return False

    return True

In [116]:
num = 0
for id in selectedList:
        
    if checkCondition(id,data,dataset) == False:
        continue
        
    Xa = util.getX(id,data,dataset)
    if Xa is None:
#        print('X is none')
        continue
    Ya = np.array(data[id]).reshape(-1,1)
#    print(Ya.shape)

    num += 1
    print(num, id, '-------------------------')
    Xn = np.c_[Xa[:1438],Xa[1:1439]]
#    Xn = np.c_[Xa[:1437],Xa[1:1438],Xa[2:1439]]
#    print(Xn.shape)
    Yn = Ya[2:1440]
#    Yn = Ya[3:1440]
#    print(Yn.shape)
    
    Xt = Xn[0:1000]
    Xval = Xn[1000:1437]
    Yt = Yn[0:1000]
    Yval = Yn[1000:1437]

    input_size = 10    
#    input_size = 15
#    classes = 5
    X = tf.placeholder(tf.float32, [None, input_size])
    Y = tf.placeholder(tf.int32, [None, 1])
    Y_one_hot = tf.one_hot(Y, classes)
    Y_one_hot = tf.reshape(Y_one_hot, [-1, classes])

    W1 = tf.Variable(tf.random_normal([input_size, 7]))
    #W1 = tf.get_variable("W1", shape=[5,classes], initializer=tf.contrib.layers.xavier_initializer())
    b1 = tf.Variable(tf.random_normal([7]))
    L1 = tf.nn.relu(tf.matmul(X, W1) + b1)

    W2 = tf.Variable(tf.random_normal([7, 7]))
    #W2 = tf.get_variable("W2", shape=[5,5], initializer=tf.contrib.layers.xavier_initializer())
    b2 = tf.Variable(tf.random_normal([7]))
    L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)

    W3 = tf.Variable(tf.random_normal([7, classes]))
    #W3 = tf.get_variable("W3", shape=[5,5], initializer=tf.contrib.layers.xavier_initializer())
    b3 = tf.Variable(tf.random_normal([classes]))
    hypothesis = tf.matmul(L2, W3) + b3
    
    cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis,labels=Y_one_hot)
    cost = tf.reduce_mean(cost_i)
    
    optimizer = tf.train.AdamOptimizer(learning_rate=0.1).minimize(cost)
    prediction = tf.argmax(hypothesis, 1)
    correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    for epoch in range(100):
        c, _ = sess.run([cost,optimizer], feed_dict={X:Xt,Y:Yt})
    
    trainAccuracy = sess.run(accuracy, feed_dict={X:Xt,Y:Yt})
    valAccuracy = sess.run(accuracy, feed_dict={X:Xval,Y:Yval})    
    print('Training:', trainAccuracy)
    print('Validation:', valAccuracy)

    log = id
    log += ',' + dataset[id][0]
    log += ',' + dataset[id][1]
    log += ',' + str(dataset[id][5])
    log += ',' + str(dataset[id][7])
    log += ',' + str(trainAccuracy)
    log += ',' + str(valAccuracy)
    log += '\n'
    f.write(log)
f.close()

1 Section697 -------------------------
Training: 0.763
Validation: 0.789474
2 Section754 -------------------------
Training: 0.752
Validation: 0.75286
3 Section1212 -------------------------
Training: 0.67
Validation: 0.675057
4 Section852 -------------------------
Training: 0.771
Validation: 0.775744
